In [1]:
import pandas as pd
import sklearn as sklearn
import numpy as np
import kaggle
import time
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lg
import optuna
import optuna.integration.lightgbm as lgb
import xgboost as xgb

from sklearn.pipeline import Pipeline
from pandas.core.frame import DataFrame
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from datetime import date
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import RepeatedKFold
from sklearn.svm import LinearSVR
from sklearn.svm import SVR

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Helper functions

In [3]:
def get_float_cols(df:DataFrame):
    return df.select_dtypes(include=float).columns.tolist()

def get_int_cols(df:DataFrame):
    return df.select_dtypes(include=int).columns.tolist()

def get_number_cols(df:DataFrame):
    return df.select_dtypes(np.number).columns.tolist()

def get_obj_cols(df:DataFrame):
    return list(df.select_dtypes(include=object).columns)

In [4]:
def plot_hist_float(df:DataFrame):
    df_numeric = df[get_float_cols(df)]
    df_numeric.hist(bins=100, figsize=(30,20))
    plt.show()

In [5]:
def plot_hist_int(df:DataFrame):
    df_numeric = df[get_int_cols(df)]
    df_numeric.hist(bins=100, figsize=(30,20))
    plt.show()

In [6]:
def plot_hist_categorical(df:DataFrame):
    
    fig = plt.figure(figsize=(26,150))

    for index, col in enumerate(get_obj_cols(df)):
        plt.subplot(25,2,index+1)
        sns.countplot(x=col, data=df)
        plt.ylabel('COUNT', size = 25)
        plt.xlabel(col, fontsize = 25)
        plt.xticks(size = 20, rotation = 45 )
        plt.yticks(size = 20)
        
    fig.tight_layout(pad=1.0)

In [7]:
def split_test_train(df:DataFrame):
    test, train = df[df['ind'].eq('test')], df[df['ind'].eq('train')]
    test = test.drop(['ind'], axis=1)
    train = train.drop(['ind'], axis=1)
    return test, train
    
def combine_test_train(test:DataFrame, train:DataFrame):
    combine = pd.concat([test.assign(ind='test'), train.assign(ind='train')])
    target = train['SalePrice']
    test_ids = test['Id']
    return combine, target, test_ids

In [8]:
def print_empty_values(df:DataFrame):
    col_names_with_na = list(df.isna().sum()[lambda x: x > 0].index)
    col_names_with_empty = list(df.isnull().sum()[lambda x: x > 0].index)
    result = set(col_names_with_na) | set(col_names_with_empty)    
    print('Columns with NA or empty: {0}'.format(result))

In [9]:
def get_empty_cols(df:DataFrame):
    return list(df.isnull().sum()[lambda x: x > 0].index)

## Data splitting

In [10]:
df_train = pd.read_csv('house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('house-prices-advanced-regression-techniques/test.csv')
df_combine, TARGET, TEST_IDS = combine_test_train(df_test, df_train)

In [11]:
df_combine.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,ind,SalePrice
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,test,NaN
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,test,NaN
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,test,NaN
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal,test,NaN
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal,test,NaN


## Building pipeline

In [12]:
def evaluate_model(predictions, test, test_labels):

    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    mean_error = np.mean(errors)
    
    res = { 'accuracy':accuracy, 'mean': mean_error } 
    return res

In [13]:
class Pipe:
    
    def __init__(self, funcs, **kwargs):
        self.funcs = funcs
        self.kwargs = kwargs
    
    def transform(self, df:DataFrame) -> DataFrame:
        for f in self.funcs:
            df = f(df, **self.kwargs)
            
        return df

In [14]:
class BaseExperiment:
    
    def __init__(self, pipe:Pipe, df:DataFrame, _TARGET_, _TEST_IDS_):
        self.pipe = pipe
        self.df = df
        self._TARGET_ = _TARGET_
        self._TEST_IDS_ = _TEST_IDS_
        self.regressor = self.create_regressor()
        
    def transform(self):        
        self.transformed = self.pipe.transform(self.df)
        return self.transformed 
    
    def create_regressor(self):
        raise NotImplementedError()
    
    def predict(self):
        test, train = split_test_train(self.transformed)
        
        # fit
        self.regressor.fit(train, self._TARGET_)
        
        # predict
        self.predicted = self.regressor.predict(test)
        
        return self.predicted
    
    def get_metric(self):
        test, train = split_test_train(self.transformed)      
        
        # predict train data
        pred_metric = self.regressor.predict(train)
        
        mse = mean_squared_error(self._TARGET_, pred_metric)
        rmse = np.sqrt(mse)
        
        return rmse

    def to_kaggle(self):
        named_tuple = time.localtime()
        time_string = time.strftime("%m/%d/%Y, %H:%M:%S", named_tuple)
        
        submission = pd.concat([self._TEST_IDS_, pd.Series(self.predicted, name='SalePrice')], axis=1)
        submission.to_csv('./submission.csv', index=False, header=True)
        
        !kaggle competitions submit -c house-prices-advanced-regression-techniques -f submission.csv -m "Test submission"
        
    def get_regressor(self):
        return self.regressor
        

In [15]:
class LinearExperiment(BaseExperiment):
    
    def create_regressor(self):
        return LinearRegression()

In [16]:
class DecisionTreeExperiment(BaseExperiment):
    
    def create_regressor(self):
        return DecisionTreeRegressor()
    
    def get_metric(self):
        test, train = split_test_train(self.transformed)   
        scores = cross_val_score(self.regressor, train, self._TARGET_, scoring='neg_mean_squared_error', cv=10)
        rmse = np.sqrt(-scores)
        return rmse.std(), rmse.mean()

In [17]:
class RandomForestExperiment(DecisionTreeExperiment):
    
    def create_regressor(self):
        return RandomForestRegressor()

In [18]:
class RandomForestWithGridSearchExperiment(RandomForestExperiment):
    
    best_params = { 
        'n_estimators': 400,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'max_features': 'sqrt',
        'max_depth': 90, 
        'bootstrap': False }
    
    def create_regressor(self):
        return RandomForestRegressor(**self.best_params)    
    
    def search_best_params(self):
        test, train = split_test_train(self.transformed)
        
        long_param_grid = {
            'bootstrap': [True, False],
            'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': [1, 2, 4],
            'min_samples_split': [2, 5, 10],
            'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000] }
           
        rf_random = RandomizedSearchCV(estimator = self.regressor,
                                       param_distributions = long_param_grid,
                                       n_iter = 100, 
                                       cv = 3, 
                                       verbose=2,
                                       random_state=42,
                                       n_jobs = -1)
        
        # fit
        rf_random.fit(train, self._TARGET_)

        # get best params
        self.best_params = rf_random.best_params_
                
        return self.best_params


In [19]:
class LightGBMExperiment(BaseExperiment):
    
    def create_regressor(self):
        return lg.LGBMRegressor()

In [20]:
class LightGBMExperimentWithParams(BaseExperiment):

    best_params = {
        'objective': 'regression', 
        'metric': 'rmse', 
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'seed': 42, 
        'feature_pre_filter': False, 
        'lambda_l1': 7.089889561174952e-07,
        'lambda_l2': 0.02142265423635661,
        'num_leaves': 10, 
        'feature_fraction': 0.4, 
        'bagging_fraction': 0.9990730129106954,
        'bagging_freq': 4, 
        'min_child_samples': 5}
    
    def create_regressor(self):
        return lg.LGBMRegressor(**self.best_params)
    
    def search_best_params(self):
        
        rkf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

        params = {
            "objective": "regression",
            "metric": "rmse",
            "verbosity": -1,
            "boosting_type": "gbdt",                
            "seed": 42 }
        
        test, train = split_test_train(self.transformed)
        X = train
        y = self._TARGET_
        dtrain = lgb.Dataset(X, label=y)
        
        study_tuner = optuna.create_study(direction='minimize')        
        optuna.logging.set_verbosity(optuna.logging.WARNING) 
        
        tuner = lgb.LightGBMTunerCV(params, 
                            dtrain, 
                            study=study_tuner,
                            seed = 42,
                            folds=rkf,
                            num_boost_round=1000                       
                            )

        tuner.run()        
        self.best_params = tuner.best_params        
        return tuner

In [21]:
class XGBExperiment(BaseExperiment):
    
    def create_regressor(self):
        return xgb.XGBRegressor()

In [22]:
class XGBExperimentWithParams(BaseExperiment):
    
    best_params = {
        'max_depth': 7, 
        'learning_rate': 0.07565159949506778,
        'n_estimators': 714, 
        'min_child_weight': 6,
        'gamma': 0.9324748648192458, 
        'subsample': 0.14074600746618596, 
        'colsample_bytree': 0.6983721197854935, 
        'reg_alpha': 0.10885510377662143,
        'reg_lambda': 0.7756168972298106, 
        'random_state': 356 }
    
    def create_regressor(self):
        return xgb.XGBRegressor(**self.best_params)
    
    def transform(self):        
        self.transformed = self.pipe.transform(self.df)
        
        # static for objective function
        XGBExperimentWithParams.TRANSFORM = self.transformed
        XGBExperimentWithParams.TARGET = self._TARGET_
        
        return self.transformed 
    
    def objective(trial):
        
        param = {
            'max_depth': trial.suggest_int('max_depth', 1, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
            'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma': trial.suggest_float('gamma', 0.01, 1.0),
            'subsample': trial.suggest_float('subsample', 0.01, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
            'random_state': trial.suggest_int('random_state', 1, 1000) }
        
        # split
        test, train = split_test_train(XGBExperimentWithParams.TRANSFORM)
        
        # fit
        model = xgb.XGBRegressor(**param)
        model.fit(train, XGBExperimentWithParams.TARGET)
        
        # get score
        scores = cross_val_score(model, train, XGBExperimentWithParams.TARGET, scoring='neg_mean_squared_error', cv=3)
        rmse = np.sqrt(-scores)
        
        return np.mean(rmse)
    
    def search_best_params(self):
        
        study = optuna.create_study(direction='minimize', study_name='regression')
        study.optimize(XGBExperimentWithParams.objective, n_trials=100)
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        self.best_param = study.best_param
        return study.best_params

In [23]:
class LinearSVRExperiment(BaseExperiment):
    
    def create_regressor(self):
        return LinearSVR(epsilon=1.5)

In [24]:
class SVRExperiment(BaseExperiment):
    
    def create_regressor(self):
        return SVR(kernel='poly', degree=2, C=100, epsilon=0.1)

In [25]:
class CatBoostExperiment(BaseExperiment):
    
    def create_regressor(self):
        return CatBoostRegressor()

In [26]:
class CatBoostWithParamsExperiment(BaseExperiment):
    
    best_params =  {'iterations': 250, 
                    'learning_rate': 0.1736687160463188,
                    'depth': 11}
    
    
    def create_regressor(self):
        return CatBoostRegressor(**self.best_params)

    def transform(self):        
        self.transformed = self.pipe.transform(self.df)

        # static for objective function
        CatBoostWithParamsExperiment.TRANSFORM = self.transformed
        CatBoostWithParamsExperiment.TARGET = self._TARGET_

        return self.transformed 
    
    def objective(trial):
        
        param = {
            'iterations': trial.suggest_int('iterations', 1, 1000),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
            'depth': trial.suggest_int('depth', 2, 16) }

        # split
        test, train = split_test_train(CatBoostWithParamsExperiment.TRANSFORM)
        
        # fit
        model = CatBoostRegressor(**param)
        model.fit(train, CatBoostWithParamsExperiment.TARGET)
        
        # get score
        scores = cross_val_score(model, train, CatBoostWithParamsExperiment.TARGET, scoring='neg_mean_squared_error', cv=3)
        rmse = np.sqrt(-scores)        
        return np.mean(rmse)
    
    def search_best_params(self):
        
        study = optuna.create_study(direction='minimize', study_name='regression')
        study.optimize(CatBoostWithParamsExperiment.objective, n_trials=50)
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        self.best_param = study.best_param
        return study.best_params

In [27]:
def impute_categorical_with_na_avail(df:DataFrame, **kwargs) -> DataFrame:
        
    cols_to_impute = [
        'Alley',
        'BsmtQual',
        'BsmtCond',
        'BsmtExposure',
        'BsmtFinType1',
        'BsmtFinType2',
        'FireplaceQu',
        'GarageType',
        'GarageFinish',
        'GarageQual',
        'GarageCond',
        'PoolQC',
        'Fence',
        'MiscFeature']
    
    for col in cols_to_impute:
        df[col] = df[col].fillna('NA')
        
    return df

In [28]:
def impute_categorical_with_na_not_avail(df:DataFrame, **kwargs) -> DataFrame:
        
    cols_to_impute = [ 
        'MSZoning',
        'Utilities',
        'Exterior1st',
        'Exterior2nd',
        'MasVnrType',
        'Electrical',
        'KitchenQual',
        'Functional',
        'SaleType']
    
    for col in cols_to_impute:
        df[col] = df[col].mode()[0]
        
    return df

In [29]:
def delete_not_needed(df:DataFrame, **kwargs) -> DataFrame:
    to_delete = [ 'Id', 'SalePrice' ]
    for d in to_delete:
        if d in df.columns:
            df = df.drop(columns=[d])
            
    return df

In [30]:
def add_features(df:DataFrame, **kwargs) -> DataFrame:
    df["SqFtPerRoom"] = df["GrLivArea"] / (df["TotRmsAbvGrd"] + df["FullBath"] + df["HalfBath"] + df["KitchenAbvGr"])
    df['Total_Home_Quality'] = df['OverallQual'] + df['OverallCond']
    df['Total_Bathrooms'] = (df['FullBath'] + (0.5 * df['HalfBath']) + df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath']))
    df["HighQualSF"] = df["1stFlrSF"] + df["2ndFlrSF"]
    return df

In [31]:
def remove_skew(df:DataFrame, **kwargs) -> DataFrame:
    
    skew_df = pd.DataFrame(get_number_cols(df), columns=['Feature'])
    skew_df['Skew'] = skew_df['Feature'].apply(lambda feature: scipy.stats.skew(df[feature]))
    skew_df['Absolute Skew'] = skew_df['Skew'].apply(abs)
    skew_df['Skewed'] = skew_df['Absolute Skew'].apply(lambda x: True if x >= 0.5 else False)

    for column in skew_df.query("Skewed == True")['Feature'].values:
        df[column] = np.log1p(df[column])
    
    return df

In [32]:
def fix_subclass(df:DataFrame, **kwargs) -> DataFrame:
    df['MSSubClass'] = df['MSSubClass'].astype(str)
    return df

In [33]:
def fix_month_sold(df:DataFrame, **kwargs) -> DataFrame:
    df['MoSold'] = (-np.cos(df['MoSold']))
    return df

In [34]:
def scale(df:DataFrame, **kwargs) -> DataFrame:
    
    scaling_type = kwargs['scaling_type']
    
    if (scaling_type == 'MinMaxScaler'):
        scaler = MinMaxScaler()
    
    if (scaling_type == 'MaxAbsScaler'):
        scaler = MaxAbsScaler()
    
    if (scaling_type == 'StandardScaler'):
        scaler = StandardScaler()
    
    if (scaling_type == 'RobustScaler'):
        scaler = RobustScaler()        
    
    # save ind column
    test_train_mask = df['ind']
    df = df.drop(['ind'], axis=1)
    
    scaler.fit(df)
    df = pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns) 
    
    # restore column
    df['ind'] = test_train_mask
    
    return df

In [35]:
def one_hot_encoding(df:DataFrame, **kwargs) -> DataFrame:
    
    test_train_mask = df['ind']
    df = df.drop(['ind'], axis=1)
    df = pd.get_dummies(df)
    df['ind'] = test_train_mask
    return df

In [36]:
def impute_numeric_cols(df:DataFrame, **kwargs) -> DataFrame:
    
    impute_method = kwargs['impute_method']
    
    col_names = [
        'LotFrontage',
        'MasVnrArea',
        'BsmtFinSF1',
        'BsmtFinSF2',
        'BsmtUnfSF',
        'TotalBsmtSF',
        'BsmtFullBath',
        'BsmtHalfBath',
        'GarageYrBlt',
        'GarageCars',
        'GarageArea' ]
    
    # all possible columns
    initial_cols = list(df.columns)
    
    for col_name in col_names:

        non_empty_numeric =  (set(get_number_cols(df)) - set(get_empty_cols(df))) | {col_name}        
        cols_names_to_drop = list((set(df.columns) - non_empty_numeric))
        
        if (kwargs['verbose']==True): print('Deleting: {0}. Imputing: {1}'.format(cols_names_to_drop, col_name))
        
        # save temp
        temp = df[cols_names_to_drop]

        # clear dataset 
        df = df.drop(columns=cols_names_to_drop)
        
        known = df.loc[ df[col_name].notnull() ]        
        unknown = df.loc[ df[col_name].isnull() ]
        
        # nothing to predict
        if (len(unknown) == 0): 
            if (kwargs['verbose']==True): print('Nothing to predict - continue')
            continue
        
        column_index = list(df.columns).index(col_name)

        all_indices = [i for i in range(unknown.shape[1])]
        diff = list(set(all_indices) - {column_index})

        y = known.values[:, column_index]
        X = known.values[:, diff]

        # predict
        if impute_method == 'randomforest':
            regressor = RandomForestRegressor(n_estimators=100, n_jobs=-1)
        if impute_method == 'knn':
            regressor = KNeighborsRegressor()
            
        regressor.fit(X, y)
        predicted = regressor.predict(unknown.values[:, diff])
        
        if (kwargs['verbose']==True): print('{0} was predicted. Len: {1}'.format(col_name, len(predicted)))
        
        # fill missings
        df.loc[ (df[col_name].isnull()), col_name ] = predicted
        
        # restore dataset
        df[cols_names_to_drop] = temp
    
    # reorder columns back
    df = df.reindex(columns = initial_cols)
    
    return df

## Plotting

In [37]:
#plot_hist_float(df_combine)

In [38]:
#plot_hist_int(df_combine)

In [39]:
#plot_hist_categorical(df_combine)

## Defining pipes

In [40]:
random_forest_imputer_pipe = Pipe([
        fix_subclass,
        impute_categorical_with_na_avail,
        impute_categorical_with_na_not_avail,
        delete_not_needed,
        impute_numeric_cols,
        one_hot_encoding
        ],
        
        verbose = False,
        impute_method = 'randomforest'
    )

knn_pipe = Pipe([
        fix_subclass,
        impute_categorical_with_na_avail,
        impute_categorical_with_na_not_avail,
        delete_not_needed,
        impute_numeric_cols,
        one_hot_encoding
        ],
        
        verbose = False,
        impute_method = 'knn'
    )

no_skew_pipe = Pipe([
        fix_subclass,
        impute_categorical_with_na_avail,
        impute_categorical_with_na_not_avail,
        delete_not_needed,
        impute_numeric_cols,
        one_hot_encoding,
        remove_skew
        ],
        
        verbose = False,
        impute_method = 'knn'
    )

no_skew_scaling = Pipe([
        fix_subclass,
        impute_categorical_with_na_avail,
        impute_categorical_with_na_not_avail,
        delete_not_needed,
        impute_numeric_cols,
        one_hot_encoding,
        remove_skew,
        scale
        ],
        
        verbose = False,
        impute_method = 'knn',
        scaling_type='RobustScaler'
    )

no_skew_scaling_cosine = Pipe([
        fix_subclass,
        fix_month_sold,
        impute_categorical_with_na_avail,
        impute_categorical_with_na_not_avail,
        delete_not_needed,
        impute_numeric_cols,
        one_hot_encoding,
        remove_skew,
        scale
        ],
        
        verbose = False,
        impute_method = 'knn',
        scaling_type='RobustScaler'
    )

no_skew_scaling_cosine_more_features = Pipe([
        fix_subclass,
        fix_month_sold,
        impute_categorical_with_na_avail,
        impute_categorical_with_na_not_avail,
        delete_not_needed,
        impute_numeric_cols,
        add_features,
        one_hot_encoding,
        remove_skew,
        scale
        ],
        
        verbose = False,
        impute_method = 'knn',
        scaling_type='RobustScaler'
    )


## Liniear regression

In [41]:
#experiment = LinearExperiment(random_forest_imputer_pipe, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

In [42]:
#experiment = LinearExperiment(knn_pipe, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

In [43]:
#experiment = LinearExperiment(no_skew_pipe, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

In [44]:
#experiment = LinearExperiment(no_skew_scaling, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

## Decision Tree

In [45]:
#experiment = DecisionTreeExperiment(no_skew_pipe, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

In [46]:
#experiment = DecisionTreeExperiment(no_skew_scaling, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

## Random Forest

In [47]:
#experiment = RandomForestExperiment(no_skew_scaling, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

In [48]:
#experiment = RandomForestExperiment(no_skew_scaling_cosine, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

In [49]:
#experiment = RandomForestExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

## Random Forest with best params

In [50]:
#experiment = RandomForestWithGridSearchExperiment(no_skew_scaling, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

In [51]:
#experiment = RandomForestWithGridSearchExperiment(no_skew_scaling_cosine, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

In [52]:
#experiment = RandomForestWithGridSearchExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

## Search params for RandomForest

In [53]:
#experiment = RandomForestWithGridSearchExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#p = experiment.search_best_params()
#print(p)

## LightGBM

In [54]:
#experiment = LightGBMExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))
#experiment.to_kaggle()

In [55]:
#experiment = LightGBMExperimentWithParams(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

## XGB with no params

In [56]:
#experiment = XGBExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))
#experiment.to_kaggle()

In [57]:
#experiment = XGBExperimentWithParams(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

## SVR

In [58]:
#experiment = LinearSVRExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

In [59]:
#experiment = SVRExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

## CatBoost

In [60]:
experiment = CatBoostExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
experiment.transform()
predicted = experiment.predict()
metric = experiment.get_metric()
print('RMSE: {0}'.format(metric))

Learning rate set to 0.043466
0:	learn: 76997.8553600	total: 270ms	remaining: 4m 29s
1:	learn: 74923.2863072	total: 283ms	remaining: 2m 21s
2:	learn: 72855.8676566	total: 294ms	remaining: 1m 37s
3:	learn: 70793.9966318	total: 316ms	remaining: 1m 18s
4:	learn: 68882.5397143	total: 324ms	remaining: 1m 4s
5:	learn: 67234.4424514	total: 350ms	remaining: 57.9s
6:	learn: 65553.9489100	total: 363ms	remaining: 51.5s
7:	learn: 63887.2534887	total: 376ms	remaining: 46.6s
8:	learn: 62301.7246945	total: 389ms	remaining: 42.8s
9:	learn: 60821.6354817	total: 400ms	remaining: 39.6s
10:	learn: 59460.2789640	total: 455ms	remaining: 40.9s
11:	learn: 58034.7926525	total: 497ms	remaining: 40.9s
12:	learn: 56632.4724103	total: 512ms	remaining: 38.8s
13:	learn: 55272.8385588	total: 534ms	remaining: 37.6s
14:	learn: 53920.8350378	total: 560ms	remaining: 36.7s
15:	learn: 52742.3189254	total: 570ms	remaining: 35s
16:	learn: 51607.5506935	total: 607ms	remaining: 35.1s
17:	learn: 50445.1386105	total: 637ms	remai

154:	learn: 18486.1111502	total: 2.94s	remaining: 16s
155:	learn: 18443.6702122	total: 2.96s	remaining: 16s
156:	learn: 18430.9001567	total: 2.98s	remaining: 16s
157:	learn: 18404.7367526	total: 2.99s	remaining: 15.9s
158:	learn: 18361.6353515	total: 3s	remaining: 15.9s
159:	learn: 18313.9046432	total: 3.01s	remaining: 15.8s
160:	learn: 18277.4733503	total: 3.02s	remaining: 15.8s
161:	learn: 18235.5603513	total: 3.04s	remaining: 15.7s
162:	learn: 18171.7619855	total: 3.05s	remaining: 15.7s
163:	learn: 18127.3358381	total: 3.06s	remaining: 15.6s
164:	learn: 18074.3995620	total: 3.07s	remaining: 15.5s
165:	learn: 18037.0041481	total: 3.08s	remaining: 15.5s
166:	learn: 18002.5728704	total: 3.1s	remaining: 15.4s
167:	learn: 17937.3928986	total: 3.11s	remaining: 15.4s
168:	learn: 17896.5310097	total: 3.13s	remaining: 15.4s
169:	learn: 17883.6285701	total: 3.14s	remaining: 15.3s
170:	learn: 17845.1726353	total: 3.15s	remaining: 15.3s
171:	learn: 17803.0929364	total: 3.17s	remaining: 15.2s
17

310:	learn: 14019.0922470	total: 4.69s	remaining: 10.4s
311:	learn: 14009.0299541	total: 4.71s	remaining: 10.4s
312:	learn: 13982.4600235	total: 4.72s	remaining: 10.4s
313:	learn: 13958.3249162	total: 4.74s	remaining: 10.4s
314:	learn: 13950.5743441	total: 4.75s	remaining: 10.3s
315:	learn: 13927.6751941	total: 4.76s	remaining: 10.3s
316:	learn: 13923.5750110	total: 4.82s	remaining: 10.4s
317:	learn: 13904.6623933	total: 4.84s	remaining: 10.4s
318:	learn: 13882.0483043	total: 4.86s	remaining: 10.4s
319:	learn: 13863.2963276	total: 4.89s	remaining: 10.4s
320:	learn: 13855.9864225	total: 4.92s	remaining: 10.4s
321:	learn: 13826.2802513	total: 4.96s	remaining: 10.4s
322:	learn: 13801.0961007	total: 4.98s	remaining: 10.4s
323:	learn: 13779.0296510	total: 5.01s	remaining: 10.4s
324:	learn: 13740.7796471	total: 5.04s	remaining: 10.5s
325:	learn: 13728.3784304	total: 5.06s	remaining: 10.5s
326:	learn: 13724.6583043	total: 5.08s	remaining: 10.5s
327:	learn: 13691.1817977	total: 5.11s	remaining

459:	learn: 11334.2987894	total: 8.11s	remaining: 9.52s
460:	learn: 11313.1596485	total: 8.13s	remaining: 9.5s
461:	learn: 11302.5748000	total: 8.15s	remaining: 9.49s
462:	learn: 11292.8271959	total: 8.16s	remaining: 9.46s
463:	learn: 11290.7238578	total: 8.17s	remaining: 9.44s
464:	learn: 11263.3141283	total: 8.19s	remaining: 9.42s
465:	learn: 11261.8476863	total: 8.2s	remaining: 9.39s
466:	learn: 11246.6034492	total: 8.21s	remaining: 9.37s
467:	learn: 11226.8873452	total: 8.22s	remaining: 9.35s
468:	learn: 11217.0169065	total: 8.23s	remaining: 9.32s
469:	learn: 11209.9932602	total: 8.24s	remaining: 9.3s
470:	learn: 11199.6130099	total: 8.26s	remaining: 9.27s
471:	learn: 11191.0290348	total: 8.27s	remaining: 9.25s
472:	learn: 11180.6404300	total: 8.28s	remaining: 9.22s
473:	learn: 11167.1633168	total: 8.29s	remaining: 9.2s
474:	learn: 11143.6990343	total: 8.38s	remaining: 9.26s
475:	learn: 11125.0780846	total: 8.44s	remaining: 9.29s
476:	learn: 11111.0075062	total: 8.53s	remaining: 9.

619:	learn: 9250.1076970	total: 11.6s	remaining: 7.11s
620:	learn: 9231.0672829	total: 11.6s	remaining: 7.09s
621:	learn: 9221.3892668	total: 11.6s	remaining: 7.07s
622:	learn: 9215.3612011	total: 11.6s	remaining: 7.04s
623:	learn: 9203.5557714	total: 11.7s	remaining: 7.03s
624:	learn: 9190.4285154	total: 11.7s	remaining: 7s
625:	learn: 9182.1955431	total: 11.7s	remaining: 6.99s
626:	learn: 9168.4378668	total: 11.7s	remaining: 6.97s
627:	learn: 9157.4158926	total: 11.7s	remaining: 6.95s
628:	learn: 9149.1594421	total: 11.8s	remaining: 6.93s
629:	learn: 9139.4097240	total: 11.8s	remaining: 6.91s
630:	learn: 9124.2755020	total: 11.8s	remaining: 6.89s
631:	learn: 9123.2549771	total: 11.8s	remaining: 6.87s
632:	learn: 9105.7956788	total: 11.8s	remaining: 6.86s
633:	learn: 9085.2379648	total: 11.8s	remaining: 6.83s
634:	learn: 9078.0164943	total: 11.8s	remaining: 6.81s
635:	learn: 9061.1239942	total: 11.9s	remaining: 6.79s
636:	learn: 9052.8636094	total: 11.9s	remaining: 6.77s
637:	learn: 9

778:	learn: 7868.0755981	total: 14.5s	remaining: 4.11s
779:	learn: 7859.2005200	total: 14.5s	remaining: 4.09s
780:	learn: 7858.3362341	total: 14.5s	remaining: 4.07s
781:	learn: 7852.9411844	total: 14.5s	remaining: 4.05s
782:	learn: 7851.1089862	total: 14.5s	remaining: 4.03s
783:	learn: 7844.3992756	total: 14.6s	remaining: 4.01s
784:	learn: 7843.7715824	total: 14.6s	remaining: 3.99s
785:	learn: 7836.4131622	total: 14.6s	remaining: 3.97s
786:	learn: 7830.1121557	total: 14.6s	remaining: 3.95s
787:	learn: 7825.0280689	total: 14.6s	remaining: 3.93s
788:	learn: 7817.7548451	total: 14.6s	remaining: 3.91s
789:	learn: 7812.3446199	total: 14.6s	remaining: 3.89s
790:	learn: 7805.4009982	total: 14.7s	remaining: 3.87s
791:	learn: 7800.1604288	total: 14.7s	remaining: 3.85s
792:	learn: 7784.7613472	total: 14.7s	remaining: 3.83s
793:	learn: 7777.4213887	total: 14.7s	remaining: 3.81s
794:	learn: 7769.4948717	total: 14.7s	remaining: 3.79s
795:	learn: 7760.3200347	total: 14.7s	remaining: 3.77s
796:	learn

936:	learn: 6801.0957783	total: 16.4s	remaining: 1.1s
937:	learn: 6799.4863384	total: 16.4s	remaining: 1.08s
938:	learn: 6794.0229815	total: 16.4s	remaining: 1.07s
939:	learn: 6793.5528445	total: 16.4s	remaining: 1.05s
940:	learn: 6784.2860521	total: 16.4s	remaining: 1.03s
941:	learn: 6780.5789008	total: 16.5s	remaining: 1.01s
942:	learn: 6769.7468644	total: 16.5s	remaining: 997ms
943:	learn: 6764.9582482	total: 16.5s	remaining: 981ms
944:	learn: 6757.4197498	total: 16.6s	remaining: 964ms
945:	learn: 6750.2403795	total: 16.6s	remaining: 946ms
946:	learn: 6743.9054177	total: 16.6s	remaining: 929ms
947:	learn: 6735.2253167	total: 16.6s	remaining: 911ms
948:	learn: 6729.6001574	total: 16.6s	remaining: 893ms
949:	learn: 6724.4065053	total: 16.6s	remaining: 875ms
950:	learn: 6717.5895781	total: 16.6s	remaining: 857ms
951:	learn: 6708.9654496	total: 16.6s	remaining: 839ms
952:	learn: 6700.7343932	total: 16.7s	remaining: 822ms
953:	learn: 6689.6206251	total: 16.7s	remaining: 804ms
954:	learn:

In [61]:
experiment = CatBoostWithParamsExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
experiment.transform()
predicted = experiment.predict()
metric = experiment.get_metric()
print('RMSE: {0}'.format(metric))
experiment.to_kaggle()